In [ ]:
import logging
from pathlib import Path

from dotenv import dotenv_values

from generator_drainage_units.utils.general_functions import remove_z_dims

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
logging.basicConfig(level=logging.DEBUG)

config = dotenv_values("..\\.env")
base_dir = config["BASE_DIR"]
# case_name = "vallei_en_veluwe"
# case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
case_name = "Leuvenumse_beek"

In [ ]:
case_path = Path(base_dir, case_name)

In [ ]:
import pandas as pd
import geopandas as gpd

hydroobjecten = gpd.read_file(Path(case_path, '0_basisdata', 'hydroobjecten.gpkg'))

In [ ]:
hydroobjecten = hydroobjecten.rename(columns={'CODE': 'code'})

In [ ]:
hydroobjecten = remove_z_dims(hydroobjecten)

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString, Point
from shapely.ops import nearest_points

In [ ]:
def get_endpoints_from_lines(lines: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Extract all unique endpoints of line features from vector data

    Args:
        lines (gpd.GeoDataFrame): GeoDataFrame containing line features

    Returns:
        gpd.GeoDataFrame: GeoDataFrame containing all unique endpoints from
        line features
    """
    lines[["startpoint", "endpoint"]] = lines["geometry"].apply(
        lambda x: pd.Series([x.coords[0], x.coords[-1]])
    )
    endpoints = pd.unique(lines[["startpoint", "endpoint"]].values.ravel("K"))
    endpoints = gpd.GeoDataFrame({"coordinates": endpoints})
    endpoints["starting_lines"] = endpoints["coordinates"].apply(
        lambda x: lines["code"][lines["startpoint"] == x].values
    )
    endpoints["ending_lines"] = endpoints["coordinates"].apply(
        lambda x: lines["code"][lines["endpoint"] == x].values
    )
    endpoints["starting_line_count"] = endpoints.apply(
        lambda x: len(list(x["starting_lines"])), axis=1
    )
    endpoints["ending_line_count"] = endpoints.apply(
        lambda x: len(list(x["ending_lines"])), axis=1
    )
    endpoints["connected_line_count"] = endpoints.apply(
        lambda x: x["starting_line_count"] + x["ending_line_count"], axis=1
    )
    endpoints_geometry = endpoints.coordinates.apply(lambda x: Point(x))
    endpoints = endpoints.set_geometry(endpoints_geometry)
    return endpoints

In [ ]:
endpoints = get_endpoints_from_lines(hydroobjecten)

In [ ]:
endpoints['ID'] = endpoints.index
endpoints = endpoints[(endpoints['starting_line_count'] == 0) | (endpoints['ending_line_count'] == 0)]
endpoints = endpoints.rename(columns={'coordinates': 'geometry'})
endpoints = gpd.GeoDataFrame(endpoints, geometry='geometry', crs=28992)
buffer_distance = 0.05
original_geometry = endpoints[['ID', 'geometry']].copy()
endpoints['geometry'] = endpoints.buffer(buffer_distance)

endpoints = endpoints.to_crs(28992)
original_geometry = original_geometry.to_crs(28992)

In [ ]:
# Perform a spatial join to find endpoints within the buffer
joined_points = gpd.sjoin(original_geometry, endpoints, how="inner", predicate="intersects")

# Filter out the endpoints with the same ID
unconnected_endpoints_points = joined_points[joined_points['ID_left'] != joined_points['ID_right']]

In [ ]:
unconnected_endpoints_points

In [ ]:
merged_df = unconnected_endpoints_points.merge(unconnected_endpoints_points, left_on='ID_left', right_on='ID_right', suffixes=('_left', '_right'))

In [ ]:
to_connect = merged_df[(merged_df['ending_lines_left'].apply(lambda x: len(x) > 0) & merged_df['starting_lines_right'].apply(lambda x: len(x) > 0))]

In [ ]:
to_connect

In [ ]:
joined_lines = gpd.sjoin(endpoints, hydroobjecten, how="left", predicate="crosses")
joined_lines = joined_lines[
    joined_lines.apply(lambda x: x['code'] not in x['starting_lines'] 
                    and x['code'] not in x['ending_lines'], axis=1)
]

In [ ]:
joined_lines

In [ ]:
unconnected_endpoints

In [ ]:

# Drop the temporary columns created by the spatial join
unconnected_endpoints = unconnected_endpoints[['ID_left', 'geometry_left']].rename(columns={'ID_left': 'ID', 'geometry_left': 'geometry'})

In [ ]:
snapped_lines = []
for line in hydroobjecten.geometry:
    snapped_line = snap_to_nearest_line(line, hydroobjecten, buffer_distance=0.05)
    snapped_lines.append(snapped_line)


In [ ]:
snapped_gdf = gpd.GeoDataFrame(geometry=snapped_lines, crs='EPSG:28992')

In [ ]:
hydroobjecten.to_file(Path(case_path, '1_tussenresultaat', 'test.gpkg'))